# Practical Session 4: ROS introduction

## Laboratorio de Robótica 
### Grado en Ingeniería Electrónica, Mecatrónica y Robótica
### Universidad de Sevilla

## Objectives

In this practical session, we will learn the basics of ROS (Robot Operating System). ROS is a set of software libraries and tools that help you build robot applications, including hardware drivers, state-of-the-art algorithms, robotics simulators, powerful developer tools, and much more. ROS is open-source and has a strong support from the robotics community, so you can find a wide spectrum of available packages for your robotics application.

In ROS, code is structured into _packages_, which are the smallest software units within a ROS system. These packages are then grouped into _workspaces_ in order to build them together. Typically, each project you work on will have its own workspace containing multiple packages related to that project. A project can also make use of packages from different workspaces, as long as those workspaces are included in the ROS system path.

In this session, we will use our Docker container to run ROS and learn a bit more about:

+ ROS basic concepts and commands.
+ What ROS workspaces are, how they are organized, and how to create them.
+ What ROS packages are, how they are organized, and how to create them.
+ How to create ROS messages and services.
+ How to create ROS nodes using the Python client library.
+ How to use some helpful ROS tools for debugging and data visualization.

## ROS basics

ROS is an open-source, meta-operating system for your robot. It provides the services you would expect from an operating system, including hardware abstraction, low-level device control, implementation of commonly-used functionality, message-passing between processes, and package management. It also provides tools and libraries for obtaining, building, writing, and running code across multiple computers. 

ROS is modular and distributed, and creates robot systems made up of multiple thin processes called **_nodes_** (potentially distributed across machines) that can communicate through the ROS communication infrastructure. ROS implements several different styles of communication, including synchronous request/reply communication over **_services_**, asynchronous streaming of data over **_topics_**, and storage of data on a **_Parameter Server_**. You can learn about all of these concepts in the [ROS conceptual overview](https://wiki.ros.org/ROS/Introduction).


### ROS commands

#### ROS Master

The ROS Master can be launched with the `roscore` command. This command runs a collection of programs to set up the ROS Master and the Parameter Server. Therefore, `roscore` must be run before any communication happen in your ROS system, as the ROS Master is the one enabling node, topic and service registration. 

#### Navigating the ROS file system

The `rospack` command allows you to get information about ROS packages, it has several functionalities that you can check with the `help` option. For instance, you can find the path of an installed ROS package or you can learn about its dependencies (`depends1` returns the first-level dependencies whereas `depends` returns all dependencies recursively):

In [ ]:
rospack find turtlebot3_gazebo
rospack depends1 turtlebot3_gazebo
rospack depends turtlebot3_gazebo

The `roscd` command allows you to change directory (cd) directly to a ROS package. Thus, you can move quickly between packages without knowing their whole path. You can move directly from anywhere to a ROS package and check or modify its files:

In [ ]:
roscd turtlebot3_gazebo
ls

__Note:__ It can get tedious to type out an entire package name. Luckily, ROS tools support TAB completion. You can start typing the name of a package, topic, file, etc., and press the TAB key to autocomplete. 

#### Understanding ROS nodes

The `rosnode` command is quite helpful for debugging, as it shows information about the running ROS nodes, including their publications and subscriptions. For instance, `rosnode list` returns the names of the running nodes, whereas `rosnode info [node name]` print information about a given node, i.e., publications, subscriptions, services, etc. You can see all options with `rosnode help`. 

The `rosrun` command allows you to run a ROS node in an arbitrary package without having to cd (or roscd) there first. You need to specify the name of the package and the node to run. 

Follow [this ROS tutorial](https://wiki.ros.org/ROS/Tutorials/UnderstandingNodes) to better understand the use of `rosnode` and `rosrun`. 

#### Understanding ROS topics

Another command quite helpful for debugging is `rostopic`, which gives information about the active topics, who is publishing or subscribed, which messages are being published, and it can even allow you to publish messages on a topic for testing. You can see all options with `rostopic help`. For instance, `rostopic type [topic name]` indicates the type of the messages that are sent through a given topic. Then `rosmsg show [message type]` can be used to provide details about that particular message.    

The `rqt` ROS package provides a set of graphical tools to display information about nodes, topics and their interconnections. For instance, the `rqt_graph` command allows you to visualize the ROS computation graph, i.e., the running nodes, their interconnections and the name of the existing topics. Moreover, the `rqt_plot` command allows you to create online plots with the data being published in certain topics.
 
Follow [this ROS tutorial](https://wiki.ros.org/ROS/Tutorials/UnderstandingTopics) to better understand the use of `rostopic`, `rosmsg` and `rqt` tools.  

#### Understanding ROS services and parameters

The `rosservice` command can be used to provide information about the existing ROS services. You can see all options with `rosservice help`. For instance, this command is helpful to list all the active services, to check the type of a given service (i.e., the data types used to call the service and get a reply), or to call a service. In a similar way as `rosmsg`, once you know the data type of a service, you can use `rossrv show [service type]` to show information about the data structure of the request and reply messages of that service. 

The `rosparam` command is used to access the parameters in the ROS Parameter Server. Among other things, you can list the parameters, and get or set the value of a given parameter. The Parameter Server can store integers, floats, boolean, dictionaries, and lists. `rosparam` uses the YAML markup language for syntax. 

Follow [this ROS tutorial](https://wiki.ros.org/ROS/Tutorials/UnderstandingServicesParams) to better understand the use of `rosservice`, `rossrv` and `rosparam`. 

__Exercise 1__

Now that you know how to use commands to inspect ROS nodes, topics, services and parameters, apply that knowledge to play with the Turtlebot3 Gazebo simulation that you ran at the beginning. As you did with the `turtlesim` node in the previuos ROS tutorials, find out the topic and service names and types to move the Turtlebot3 in circles and, after that, reset the simulation to take the robot back to its initial position.   


## ROS workspaces

ROS packages can be built as standalone projects, but the natural way to work with them is by integrating them into __*workspaces*__. A ROS workspace is a folder which can contain multiple packages that are combined together to build more complex projects. The packages in a workspace can be built and installed together all at once. In order to deal with workspace and package compilation, we will use __catkin__, which is the official build system of ROS. A build system is responsible for generating _targets_ from raw source code that can be used by an end user. These targets may be in the form of libraries, executable programs, generated scripts, exported interfaces (e.g. C++ header files) or anything else that is not static code. In ROS terminology, source code is organized into __packages__, with each package typically consisting of one or more targets when built. More specifically, catkin builds on top of CMake by combining CMake macros with Python scripts to add functionality beyond CMake's standard workflow. catkin's workflow is very similar to CMake's but it adds support for automatic package discovery and the ability to build multiple interdependent projects simultaneously.

Coming back to workspaces, the following is the recommended and typical catkin workspace layout:

<figure style="text-align:center">
  <img src="images/workspace_structure.png" alt="" width=700>
  <figcaption>Fig. 1: ROS workspace layout.</figcaption>
</figure>
 
A catkin workspace can contain up to four distinct spaces, each serving a different role in the software development process:

+ __Source space:__ It contains the source code of catkin packages. This is where you can extract/checkout/clone source code for the packages you want to build. Each folder within the source space contains one or more catkin packages. This space should remain unchanged by configuring, building, or installing. The root of the source space contains a symbolic link to the catkin's 'toplevel' `CMakeLists.txt` file. This file is invoked by CMake during the configuration of the catkin projects in the workspace and it is created when the workspace is initialized.

+ __Build space:__ This is where CMake is invoked to build the catkin packages in the source space. CMake and catkin keep their cache information and other intermediate files here. 

+ __Development (Devel) space:__ This is where built targets are placed prior to being installed. The way targets are organized in the devel space is the same as their layout when they are installed. This provides a useful testing and development environment which does not require invoking the installation step. 

+ __Install space:__ Once targets are built, they can be installed into the install space by invoking the install target, usually with `make install`.


A catkin workspace called `catkin_ws` can be created with the following commands:

In [ ]:
mkdir -p catkin_ws/src
cd catkin_ws/
catkin_make

Running `catkin_make` the first time in your workspace will create the `CMakeLists.txt` symbolic link in your `src` folder. Additionally, if you check your current directory, you should now see a `build` and `devel` folder. Inside the `devel` folder you can see that there are several `setup.*sh` files. Sourcing any of these files will overlay this workspace on top of your environment, i.e., it will include the path of your workspace in the `ROS_PACKAGE_PATH` environment variable, which is used to locate ROS packages, nodes, messages, and other componentes in your ROS system. You can source your workspace with the command `source devel/setup.bash`. After sourcing, check the ROS package path with `echo $ROS_PACKAGE_PATH`, you'll see that your new workspace is now included.

Each time you want to build your workspace, you need to call `catkin_make` from the top level directory of your catkin workspace. This will build any packages in the source space (`~/catkin_ws/src`) to the build space (`~/catkin_ws/build`). Any source files, python libraries, scripts or any other static files will remain in the source space. However, any generated files such as libraries, executables, or generated code will be placed in the devel space.

__Note:__ Recall that `catkin_make` must always be called from the root folder of a workspace.



__Exercise 2__ 

Go to your sharded directory `lab_rob_shared` and create there a catkin workspace. After that, on your host machine, you should see the workspace you just created in the `lab_rob_shared` folder. From a terminal on your host machine, clone your Git repository for this course into the `src` directory of your workspace. This way, all your code will be accessible within the Docker container's ROS workspace. __Important__: clone the repository you created in the first practical session for your group, not the original one provided by the professor for the course. 

## ROS packages

Software in ROS is organized into packages. A package is a folder that might contain ROS nodes, a ROS-independent library, a dataset, configuration files, a third-party piece of software, or anything else that logically constitutes a useful module. The goal of these packages it to provide this useful functionality in an easy-to-consume manner so that software can be easily reused.

In order to be considered a ROS package, the folder must meet a few requirements:

+ It must contain a catkin compliant `package.xml` file. This package manifest is an XML file that defines properties about the package such as the package name, version numbers, authors, maintainers, and dependencies on other catkin packages.
+ It must contain a `CMakeLists.txt` file that describes how to build the code and where to install it. The `CMakeLists.txt` file used for a catkin package follows the standard format for any CMake project but with a few additional constraints. 
+ Each package must have its own folder. This means no nested packages nor multiple packages sharing the same directory.

ROS packages tend to follow a common structure, these are some of the directories that it may contain: 

+ `src/`: Source files, i.e., the code to build the nodes and libraries of the package.
+ `scripts/`: executable scripts, this may include Python nodes. 
+ `include/package_name/`: C++ include headers.
+ `msg/`: Folder containing message files(.msg).
+ `srv/`: Folder containing service files (.srv).
+ `launch/`: Folder containing launch files (.launch).
+ `config/`: Folder containing configuration files.

You can use the `catkin_create_pkg` command to create a new package. For example, `catkin_create_pkg beginner_tutorials std_msgs rospy roscpp` will create a package called `beginner_tutorials` which depends on the packages `std_msgs`, `roscpp`, and `rospy`. These are __first-order__ dependencies of the package; in many cases, a dependency will also have its own dependencies (e.g., `rospy` has other dependencies), so the total list of dependencies of the package will typically be longer. The previous command will create a `beginner_tutorials` folder which contains a `package.xml` and a `CMakeLists.txt`, which are partially initialized. When you create a package, you will see that a minimal set of tags are always included in its `package.xml` file: 

+ `<name>` - The name of the package.
+ `<version>` - The version number of the package (required to be 3 dot-separated integers).
+ `<description>` - A description of the package contents.
+ `<maintainer>` - The name of the person(s) that is/are maintaining the package.
+ `<license>` - The software license(s) (e.g. GPL, BSD, ASL) under which the code is released.

While you develop your ROS package, you will probably go and customize your package manifest file, e.g., by updating its description, maintainers and dependencies. If you want to add a new dependency to a ROS package in its manifest file, you can specify six types of dependencies:

+ `<build_depend>` Packages that are needed to build this package. This is the case when any file from these packages is required at build time. This can be including headers from these packages at compilation time or linking against libraries from these packages.
+ `<exec_depend>` Packages that are needed to run code in this package. This is the case when you depend on shared libraries in this package.
+ `<build_export_depend>` Packages that are needed to build libraries against this package. This is the case when you transitively include their headers in public headers in this package.
+ `<depend>` It specifies that a dependency is a build, export, and execution dependency. __This is the most commonly used dependency tag.__
+ `<test_depend>` It specifies only additional dependencies for unit tests. They should never duplicate any dependencies already mentioned as build or run dependencies.
+ `<buildtool_depend>` It specifies build system tools which this package needs to build itself. Typically the only build tool needed is catkin. __All packages have at least a build tool dependency on catkin.__ 
+ `<doc_depend>` It specifies documentation tools which this package needs to generate documentation.

If you add new dependencies to your packages, apart from updating the `package.xml` file, you should also update `CMakeLists.txt`. Remember that this file tells catkin how to build your package. Therefore, build dependencies must be listed in the `find_package()` macro, in order to find them before compilation. __You should only `find_package` components that are needed at build time, you should not add execution dependencies__. Furthermore, the `catkin_package()` macro may need to be updated. This macro generates CMake config files for your package and it is specifically for building C++ code in other packages which depends on yours. More specifically, the macro specifies header directories and libraries of your package that other dependent projects would need to compile against this package. The `CATKIN_DEPENDS` parameter of the macro should include all the catkin packages that are needed (via their include directories, flags, and libraries) so that other packages that depend on this package compile against this package. In general, ROS `CMakeLists.txt` files contain numerous comments, making them easy to modify according to your specific needs. A detailed explanation of all the `CMakeLists.txt` macros can be found [here](https://wiki.ros.org/catkin/CMakeLists.txt).

__Note:__ If you add Python nodes to your package, a good practice is to place them in the `scripts` folder. Remember making these scripts executable so that they can later be run by ROS. 


__Exercise 3__ 

Navigate to the `src` folder of the workspace you just created in the sharded directory `lab_rob_shared` and create the `beginner_tutorials` package, within the folder of your previously cloned repository. The path of your package should be something like this: `catkin_workspace_name/src/lab_robotica_gID/beginner_tutorials`. Build your workspace to check everything is correct. Take a while to inspect the `package.xml` and `CMakeLists.txt` files. Then customize the description and maintainer in the manifest file of your package. Now, you have a new ROS package that should be accessible from everywhere. Navigate to a different folder and type `roscd beginner_tutorials` (recall that the TAB key should autocomplete the name) to move into your package folder. Didn't it work? Recall that ROS won't find the packages in your workspace until you _source_ it.  

__Note:__ Remember the workflow of this course. You should use your host machine for code development, file editing, and Git repository management. Then use the Docker container to build, execute, and test your code; or to use any ROS commands, e.g., to create new packages or workspaces. If you find a Git "detected dubious ownership" error in your host machine when trying to commit your changes after creating the ROS package, have a look at this possible [issue](https://confluence.atlassian.com/bbkb/git-command-returns-fatal-error-about-the-repository-being-owned-by-someone-else-1167744132.html). 

## Creating your own ROS messages and services

Your ROS nodes can send/receive data through topics and/or services. For that, they can use existing message/service types, but you could also define your own messages and services in your package. `msg` files are simple text files that describe the fields of a ROS message and they are stored in the `msg` directory of a package. An `srv` file describes a service and it is composed of two parts: a request and a response. These files are stored in the `srv` directory of the package.

Follow this [tutorial](https://wiki.ros.org/ROS/Tutorials/CreatingMsgAndSrv) to learn how to create customized messages and services in your package.


## Creating your first ROS nodes in Python

### Publisher and subscriber 

Let's create our first pair of ROS nodes in Python. For that, we'll use the `rospy` client library. We'll create a _publisher_ node whose role will be to publish messages in a certain topic, and a _subscriber_ node whose role will be reading those messages. 

Follow this [tutorial](https://wiki.ros.org/ROS/Tutorials/WritingPublisherSubscriber%28python%29) to create your publisher/subcriber ROS nodes in Python. Once you've created your nodes, run them both using `rosrun` to see how they work. Remember running `roscore` first in a different terminal. You can use `rosnode` to explore the connection between your nodes and `rostopic` to see the data being published.

__Important:__ As we are using Python 3 in our Docker container, modify the first line of both Python scripts to `#!/usr/bin/python3`.

### Service and client

Now let's create another pair of nodes in Python implementing a service. We'll create a node whose role will be offering a service that receives two ints and returns the sum, and 
another node whose role will be calling that service. 

Follow this [tutorial](https://wiki.ros.org/ROS/Tutorials/WritingServiceClient%28python%29) to create your service/client ROS nodes in Python. Once you've created your nodes, run them both using `rosrun` to see how they work. Remember running `roscore` first in a different terminal. You can use `rosnode` to check the nodes running and the service they offer, and `rosservice` to check information about the offered service and to call it.

__Important:__ As we are using Python 3 in our Docker container, modify the first line of both Python scripts to `#!/usr/bin/python3`.


__Exercise 4__ 

Once you have your `beginner_tutorials` package complete and working, update your Git repository to include it.

## Debugging and visualization tools

In a typical ROS system, multiple nodes communicate with each other, which can make debugging challenging when something goes wrong. To assist with this, ROS provides several helpful tools that allow you to visualize what's happening. Let’s introduce a couple of basic tools here.

### Using the logging console

ROS provides a logging framework through which nodes can publish different types of messages with information. These messages are categorized depending on their level of severity: _FATAL, ERROR, WARN, INFO, DEBUG_. FATAL has the highest priority, while DEBUG has the lowest. In the tutorial on writing your first ROS nodes, you already learnt how to send logging messages from a node usign the `rospy` library. These messages are printed on the screen, but you can also access them more efficiently using `rqt_console`, a tool that integrates with ROS's logging framework to display node output. If you open the console with the command `rosrun rqt_console rqt_console`, you will see a window where you can display all logging messages, pause them, or filter them by category or using other rules. 

<figure style="text-align:center">
  <img src="images/rqt_console.png" alt="" width=700>
  <figcaption>Fig. 2: ROS rqt console.</figcaption>
</figure>

By running `rosrun rqt_logger_level rqt_logger_level`, you can also set the logger level of each running node in the system. The default logger level is INFO. Setting a logger level ensures the node only outputs messages of that priority or higher. For example, if you set a node's level to WARN, you will receive all WARN, ERROR, and FATAL messages.

Now you can start a simple simulation in a new terminal with `rosrun turtlesim turtlesim_node`. If you open `rqt_console`, unless you have configured the logger level to DEBUG, you won't see many messages. Try to navigate the turtle into the wall and you will see how WARN messages are displayed: 

In [ ]:
rostopic pub /turtle1/cmd_vel geometry_msgs/Twist -r 1 -- '{linear: {x: 2.0, y: 0.0, z: 0.0}, angular: {x: 0.0,y: 0.0,z: 0.0}}'



### Recording and playing back data

ROS allows you to record published messages in a `bag` file. You can record messages and play them back later to replicate a certain behavior and debug your nodes. Follow this [tutorial](https://wiki.ros.org/ROS/Tutorials/Recording%20and%20playing%20back%20data) to learn how to record and replay bag files with the `rosbag` tool. 

### RViz

ROS has a quite powerful 3D visualization tool called RViz which can display all types of sensor data published in a ROS system. Note that RViz is just a display for data not a simulator. In fact, it could be used to show information from simulated or real robots, even simultaneously. RViz can be launched calling `rosrun rviz rviz`. 

<figure style="text-align:center">
  <img src="images/initial_rviz.png" alt="" width=800>
  <figcaption>Fig. 3: RViz window.</figcaption>
</figure>

In the initial window, the big black thing is the 3D world view (empty because there is nothing to see yet). On the left, you'll see a list of all loaded displays. At the beginning, it just contains the global options and the time view. A display is something that draws something in the 3D world, and has likely some options available in the displays list. An example is a map, a point cloud, a laser scan, a camera image stream, the robot state, etc. Each display has a type which indicates what kind of data this display will visualize, and a unique name. For example, you could have two laser scanners on your robot, and you might create two "Laser Scan" displays named "Laser Base" and "Laser Head".

For the moment, we only ran simple simulations with few data. Once you start to play with more complex ROS systems, RViz will become quite useful. You can learn more about how to use it wih the [user's guide](https://wiki.ros.org/rviz/UserGuide).

__Exercise 5__

Run a ROS simulation of the Turtlebot3 robot with the command: `roslaunch turtlebot3_gazebo turtlebot3_empty_world.launch`. Then open RViz and add a display to visualize the `/tf` topic. This topic contains the _transform tree_ with all the coordinate frames and their relations. You should set the "Fixed frame" to "odom" which is a static frame located at the initial position of the robot. In another terminal, run a ROS teleoperation interface to move the robot with the keyboard: `roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch`. You should see frames moving in RViz. 

## Summary

In this practical session, you should have learned the following:

+ The basic concepts of ROS: nodes, topics, servicies, messages.
+ How to use the basic ROS commands.
+ How to create and understand your first ROS workspace. 
+ How to create and understand your first ROS package.
+ How to create your own message and service within a ROS package.
+ How to create ROS nodes in Python and how to publish/subscribe to topics and offer/call services. 
+ How to use the `rqt_console` tool to visualize logging information and the `rosbag` tool to record and play back data. 
+ How to use RViz to visualize data in a ROS system.